Importation des librairies


In [207]:
import numpy as np

Class de definition de repère

In [208]:
# Calculer l'inverse d'une matrice 

def invert_matrix(mat):
    """
    Retourne l'inverse de `mat` ou lève une exception si non inversible.
    """
    try:
        return np.linalg.inv(mat)
    except np.linalg.LinAlgError:
        raise ValueError("Matrice singulière : impossible d'appliquer l'inverse.")

class RepereLocal:
    def __init__(self, origine, rotation_matrix=None, nom=None):
        self.origine = np.array(origine)  # Position de du repère dans W
        # Matrice de rotation 3x3 (identité si non spécifiée)
        self.rotation = rotation_matrix if rotation_matrix is not None else np.eye(3)
        self.nom = "nom" if nom is not None else "Repère Local"
    
    def get_transform_matrixs(self):
        """Retourne la matrice de transformation homogène 4x4"""
        T = np.eye(4)
        T[0:3, 0:3] = self.rotation
        T[0:3, 3] = self.origine
        T_inv = invert_matrix(T)
        return [ T , T_inv]

    



In [209]:
def transform_point_RA_to_RB(point_RA, repere_B):
        point = point_RA
        [ T , T_inv]= repere_B.get_transform_matrixs()
        point_in_C = T_inv @ point
    
        return point_in_C


In [210]:
# 1. Définir le repère monde W (repère de référence global)
Repere_W = RepereLocal(origine=[0, 0, 0], nom="Repère W")


In [211]:
Repere_C = RepereLocal(origine=[-16, 16, 20], rotation_matrix=np.array([[1/np.sqrt(2), 0.5, 0.5],
                                                                    [1/np.sqrt(2), -0.5, -0.5],
                                                                    [0., 1/np.sqrt(2), -1/np.sqrt(2)]]),
                                                                    nom="Repère C")
Repere_C.get_transform_matrixs()

[array([[  0.70710678,   0.5       ,   0.5       , -16.        ],
        [  0.70710678,  -0.5       ,  -0.5       ,  16.        ],
        [  0.        ,   0.70710678,  -0.70710678,  20.        ],
        [  0.        ,   0.        ,   0.        ,   1.        ]]),
 array([[ 0.70710678,  0.70710678,  0.        ,  0.        ],
        [ 0.5       , -0.5       ,  0.70710678,  1.85786438],
        [ 0.5       , -0.5       , -0.70710678, 30.14213562],
        [ 0.        ,  0.        ,  0.        ,  1.        ]])]

In [212]:
H = 480  # Hauteur de l'image en pixels
W = 640  # Largeur de l'image en pixels
origin_I = [H/2, W/2, 0]

In [213]:
Repere_I = RepereLocal(origine=origin_I, rotation_matrix=np.array([[-1, 0, 0],
                                                                    [0, -1, 0.],
                                                                    [0, 0, 1. ]]),
                                                                    nom="Repère_image")
Repere_I.get_transform_matrixs()

[array([[ -1.,   0.,   0., 240.],
        [  0.,  -1.,   0., 320.],
        [  0.,   0.,   1.,   0.],
        [  0.,   0.,   0.,   1.]]),
 array([[ -1.,  -0.,  -0., 240.],
        [ -0.,  -1.,  -0., 320.],
        [  0.,   0.,   1.,   0.],
        [  0.,   0.,   0.,   1.]])]

In [214]:
point_w = np.array([-2.25,11,1.5,1]).reshape(4,1)
#point.reshape(4,1)

In [215]:
point_C = transform_point_RA_to_RB(point_w, Repere_C)

point_C

array([[ 6.18718434],
       [-3.70647545],
       [22.45647545],
       [ 1.        ]])

In [216]:
# # 4. Créer un repère C dans le monde W
# # C est décalé de (2, 3, 1) et tourné de 45° autour de l'axe Z
# angle = np.radians(45)
# rotation_C = np.array([
#     [np.cos(angle), -np.sin(angle), 0],
#     [np.sin(angle), np.cos(angle), 0],
#     [0, 0, 1]
# ])
# C = RepereLocal(origine=[2, 3, 1], rotation_matrix=rotation_C)

# # 5. Définir un point M dans le repère monde W
# M_in_W = np.array([5, 7, 2])



# # 7. Calculer les coordonnées de M dans C
# M_in_C = transform_point_W_to_C(M_in_W, C)

# # Afficher les résultats
# print(f"Repère monde W: origine à {W.origine}")
# print(f"Repère C: origine à {C.origine} dans W")
# print(f"Angle de rotation C: 45° autour de Z")
# print(f"\nPoint M dans le repère W: {M_in_W}")
# print(f"Point M dans le repère C: {M_in_C}")
# print(f"\nCoordonnées arrondies: [{M_in_C[0]:.2f}, {M_in_C[1]:.2f}, {M_in_C[2]:.2f}]")

In [217]:
np.eye(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [218]:

class Camera:
    """
    Classe représentant une caméra avec sa matrice intrinsèque K.
    Permet de projeter des points 3D vers le repère caméra et l'image.
    """
    
    def __init__(self, K, R_camera=None):
        """
        Initialise la caméra.
        
        Args:
            K (np.ndarray): Matrice intrinsèque 3x3
            R (np.ndarray): Matrice de rotation 3x3 (optionnel)
            t (np.ndarray): Vecteur de translation 3x1 (optionnel)
        """
        self.K = K  # Matrice intrinsèque
        self.R_camera = R_camera if R_camera is not None else np.eye(4)  # Rotation (identité par défaut)
        #self.t = t if t is not None else np.zeros((3, 1))  # Translation (zéro par défaut)
    
    
    
    def camera_vers_image(self, point_RC):
        """
        Projette un point du repère caméra vers le plan image.
        
        Args:
            point_camera (np.ndarray): Point 3D dans le repère caméra
        
        Returns:
            np.ndarray: Coordonnées 2D dans l'image (x, y)
        """
        point_RC = point_RC.reshape(3, 1) if point_RC.ndim == 1 else point_RC
        
        # Projection perspective
        point_image_RE = self.K @ point_RC
        
        # Conversion en coordonnées inhomogènes
        point_image_RE = np.round(point_image_RE[:2] / point_image_RE[2])

        return point_image_RE.flatten()
    
    






In [219]:
fx = 800
fy = 800
s = 0
cx = 0      
cy = 0          
K = np.array([[fx, s, cx],
             [0, fy, cy],   
             [0,  0,    1]])    

R_camera =  RepereLocal(origine=[-16, 16, 20], rotation_matrix=np.array([[1/np.sqrt(2), 0.5, 0.5],
                                                                        [1/np.sqrt(2), -0.5, -0.5],
                                                                        [0., 1/np.sqrt(2), -1/np.sqrt(2)]]),
                                                                        nom="Repère Camera")

Camera1 = Camera(K, R_camera)   

In [232]:
point_RE = Camera1.camera_vers_image(np.array([1,2,3]))
point_RE


array([267., 533.])

In [231]:
point_RE = np.array([point_RE[0],point_RE[1],0,1])    
point_RE

array([267., 533.,   0.,   1.])

In [230]:
point_image = transform_point_RA_to_RB(point_RE,Repere_I)
point_image

array([ -27., -213.,    0.,    1.])

In [ ]:
angle = np.radians(45)
rotation_C = np.array([
    [np.cos(angle), -np.sin(angle), 0],
    [np.sin(angle), np.cos(angle), 0],
    [0, 0, 1]
])